In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Importa dati sulle vaccinazioni

In [2]:
df_vacc = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv')
df_vacc = df_vacc.fillna(method='backfill')

def get_vaccine_data(country):
    # source ourworldindata
    
    df_vacc_country = df_vacc[df_vacc['location']==country].iloc[2:,:]
    
    date = pd.to_datetime(df_vacc_country['date'])
    vacc1 = np.array(df_vacc_country['people_vaccinated_per_hundred'])  
    vacc2 = np.array(df_vacc_country['people_fully_vaccinated_per_hundred']) 
    
    df_vacc_new = pd.DataFrame(np.transpose([vacc1,vacc2]))
    df_vacc_new.index = date
    df_vacc_new.columns=['% vaccinated with 1 dose','% fully vaccinated']
    
    return df_vacc_new

### Importa dati epidemiologici

In [3]:
file_confirmed='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
file_deaths='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
file_recovered='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
df_confirmed=pd.read_csv(file_confirmed)
df_deaths=pd.read_csv(file_deaths)
df_recovered=pd.read_csv(file_recovered)
date = pd.to_datetime(df_confirmed.columns[4:])

def get_epidemic_data(country):
    # source Johns Hopkins Unversity
    ydata_cases = (df_confirmed[df_confirmed['Country/Region']==country].iloc[:,4:]).sum()
    ydata_deaths = (df_deaths[df_deaths['Country/Region']==country].iloc[:,4:]).sum()
    ydata_rec = (df_recovered[df_recovered['Country/Region']==country].iloc[:,4:]).sum()
    ydata_inf = ydata_cases-ydata_deaths-ydata_rec   
    daily_cases = ydata_cases.diff().rolling(window=7).mean()
    daily_deaths = ydata_deaths.diff().rolling(window=7).mean()
    
    df_epidemic = pd.DataFrame(np.transpose([ydata_cases,ydata_inf,ydata_deaths,ydata_rec,daily_cases,daily_deaths]))
    df_epidemic.index = date
    df_epidemic.columns = ['Total cases','Active infected','Total deaths','Total recovered','Daily cases (avg 7 days)','Daily deaths (avg 7 days)']
    
    return df_epidemic

### Risultati

In [7]:
get_vaccine_data('Germany')

,% vaccinated with 1 dose,% fully vaccinated
date,,
2020-12-29,0.11,0.00
2020-12-30,0.19,0.00
2020-12-31,0.25,0.00
2021-01-01,0.28,0.00
2021-01-02,0.33,0.00
...,...,...
2021-09-27,67.24,63.59
2021-09-28,67.32,63.73
2021-09-29,67.41,63.86


In [6]:
get_epidemic_data('Germany')

,Total cases,Active infected,Total deaths,Total recovered,Daily cases (avg 7 days),Daily deaths (avg 7 days)
2020-01-22,0.0,0.0,0.0,0.0,NaN,NaN
2020-01-23,0.0,0.0,0.0,0.0,NaN,NaN
2020-01-24,0.0,0.0,0.0,0.0,NaN,NaN
2020-01-25,0.0,0.0,0.0,0.0,NaN,NaN
2020-01-26,0.0,0.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...
2021-09-29,4228774.0,4135131.0,93643.0,0.0,7916.714286,57.142857
2021-09-30,4239773.0,4146058.0,93715.0,0.0,8014.428571,57.857143
2021-10-01,4249061.0,4155280.0,93781.0,0.0,8065.000000,58.714286
2021-10-02,4255543.0,4161752.0,93791.0,0.0,8073.428571,56.142857
